In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

# 连不上服务器来下载model
# fashion_mnist = keras.datasets.fashion_mnist
# (train_images, train_labels),(test_images, test_labels) = fashion_mnist.load_data()
# print(train_images.shape)

1.12.0


In [2]:
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist = input_data.read_data_sets('./Train_CNN/mnist/',one_hot=True)
# mnist = tf.data.Dataset.list_files(file_pattern='./Train_CNN/mnist/')
# (x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data(path='./Train_CNN/mnist/')
sess = tf.InteractiveSession()
# 占位符
x = tf.placeholder("float",shape=[None, 784])
y_ = tf.placeholder("float",shape=[None,10])
# 变量
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
sess.run(tf.initialize_all_variables())
# 类别预测与损失函数 -> 交叉熵
y = tf.nn.softmax(tf.matmul(x,W)+b)
cross_entropy = tf.reduce_sum(y_*tf.log(y))
# 训练模型
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
for i in range(1000):
    batch = mnist.train.next_batch(50)
    train_step.run(feed_dict={x:batch[0], y_:batch[1]})
    
# 评估模型
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))
print(accuracy.eval(feed_dict={x:mnist.test.images,y_:mnist.test.labels}))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.


URLError: <urlopen error [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。>

In [3]:
# 构建一个多层卷积网络
# 权重初始化
def weight_variables(shape):
    initial = tf.truncated_normal(shape=shape,stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# 卷积（->步长）和池化
def conv2d(x, W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1],
                          strides=[1,2,2,1],padding='SAME')

# 第一层卷积->一个卷积接一个max pooling
W_conv1 = weight_variables([5,5,1,32])  # 每5*5patch->32个特征
b_conv1 = bias_variable([32])
# x变成4D向量(2width,3height)
x_images = tf.reshape(x,[-1,28,28,1])
# 卷积，加上偏置项，应用ReLU激活函数，池化
h_conv1 = tf.nn.relu(conv2d(x_images,W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)


In [4]:
# 第二层卷积
W_conv2 = weight_variables([5,5,32,64]) # 每5*5patch->64特征
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# 密集连接层
W_fc1 = weight_variables([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


In [ ]:
# Dropout
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob=keep_prob)
# 输出层
W_fc2 = weight_variables([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
# 训练和评估模型->ADAM优化器->梯度下降
cross_entropy = -tf.reduce_sum(y_ * tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.initialize_all_variables())
for i in range(2000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:  # 每100项输出一次
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_:batch[1], keep_prob: 1.0})
        print("step:%d, training accuracy: %g"%(i, train_accuracy))
    train_step.run(feed_dict={x:batch[0], y_:batch[1], keep_prob:0.5})
    
print("test accuracy:%g"%accuracy.eval(feed_dict={
    x :mnist.test.images, y_:mnist.test.labels, keep_prob: 1.0}))

step:6600, training accuracy: 1
step:6700, training accuracy: 1
step:6800, training accuracy: 0.96
step:6900, training accuracy: 1
step:7000, training accuracy: 1
step:7100, training accuracy: 1
